First, We need to add the toolbox into the search path.

In [1]:
import sys
sys.path.append('..')

Then, before we start the simulation, we need to prepare the dataset. If there is not data in the given path, the dataset API will automatically download the correspnding data. In this example, we will use the Benchmark Dataset as example.

In [2]:
from SSVEPAnalysisToolbox.datasets import BenchmarkDataset
dataset = BenchmarkDataset(path = '2016_Tsinghua_SSVEP_database')

100%|########################################| 106M/106M [00:00<00:00, 106GB/s]
SHA256 hash of downloaded file: 2e72ca82202ad82268c45a204cc19ae9df6e4866c62b4ecd1e2a367ec14f601e
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
 53%|####################1                 | 56.0M/105M [00:15<00:13, 3.55MB/s]
C:\Users\wangz\Documents\GitHub\SSVEP-Analysis-Toolbox\demo\..\SSVEPAnalysisToolbox\datasets\basedataset.py:174: UserWarning: There is an error when donwloading 'S2' data. So retry (1) after 10 seconds.
  warnings.warn("There is an error when donwloading '{:s}' data. So retry ({:n}) after 10 seconds.".format(subject.ID, download_try_count))
100%|########################################| 105M/105M [00:00<00:00, 105GB/s]
SHA256 hash of downloaded file: 119811b9d12f6fb865b4e9436f688208344405282d9621a1691349972f0f73fa
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that 

Because EEG signals normally contain large noise, we need do preprocesses when we extract signals. Therefore, we need hook the preprocess method on the dataset. The Benchmark Dataset paper already provides the suggested preprocess methods. These method has been included in this toolbox and can be directly used.

In [3]:
from SSVEPAnalysisToolbox.utils.benchmarkpreprocess import preprocess
dataset.regist_preprocess(preprocess)

Because the filter-bank approach has been successfully adopted to improve the recognition performance in literature, we need to hook the filter-bank method on the dataset. The Benchmark Dataset paper already provides the suggested filter-bank method. This method has also been included in this toolbox and can be directly used.

In [5]:
from SSVEPAnalysisToolbox.utils.benchmarkpreprocess import filterbank
dataset.regist_filterbank(filterbank)

After preparing the dataset, we need to prepare the recognition method. The toolbox contains various methods with different implementations. This example use the eCCA method as an example to show how to use the method API. In addition, because we use the filter-bank approach, we need to predefine the weights of different filter banks. The Benchmark Dataset paper already provides the suggested weights. The method of generating these weights has been implemented in this toolbox and can be directly used.

In [9]:
from SSVEPAnalysisToolbox.utils.benchmarkpreprocess import suggested_weights_filterbank
weights_filterbank = suggested_weights_filterbank()
from SSVEPAnalysisToolbox.algorithms import ECCA
recog_model = ETRCA(weights_filterbank = weights_filterbank)

Now, we can prepare the simulation. In this example, 

1. we will only use 9 occipital channels;
2. All 40 classes in the Benchmark data are considered.
3. 5 harmonic components are considered in the SSVEP reference signals;
4. The first 1 second EEG signals after removing 0.14s latency are applied for this example;
5. Only the second subject's EEG is used for the individual recognition;
6. EEG signals in the first block is used for testing the recognition method;
7. EEG signals in other blocks is used for training the recognition method.

In [10]:
from SSVEPAnalysisToolbox.utils.benchmarkpreprocess import suggested_ch
ch_used = suggested_ch()
all_trials = [i for i in range(dataset.trial_num)]
harmonic_num = 5
tw = 1
sub_idx = 2-1
test_block_idx = 0
test_block_list, train_block_list = dataset.leave_one_block_out(block_idx = test_block_idx)

The whole simulation is divided into 2 steps:

1. Train the recognition model:
   
   1. Prepare the training materials: The training process of most recognition methods requires the training data, corresponding labels, the SSVEP reference signals (sine-cosine reference signals), and freqeucies of labels. Although the eCCA does not need freqeucies of labels, we still show how to prepare and input them.
   2. Use the training materials to train the model. We also show how to record the training time.

In [11]:
ref_sig = dataset.get_ref_sig(tw, harmonic_num)
freqs = dataset.stim_info['freqs']
X_train, Y_train = dataset.get_data(sub_idx = sub_idx,
                                    blocks = train_block_list,
                                    trials = all_trials,
                                    channels = ch_used,
                                    sig_len = tw)

In [16]:
import time
tic = time.time()
recog_model.fit(X=X_train, Y=Y_train, ref_sig=ref_sig, freqs=freqs) 
toc_train = time.time()-tic

2. Test the recognition model:

   1. Prepare the testing materials: Normally, we only need the testing EEG signals. But we also extract the corresponding testing labels for further calculating classification accuracy;
   2. Use the testing materials to test the model. We also record the testing time and compute the averaged testing time of each trial for further calculating the ITR.

In [18]:
X_test, Y_test = dataset.get_data(sub_idx = sub_idx,
                                    blocks = test_block_list,
                                    trials = all_trials,
                                    channels = ch_used,
                                    sig_len = tw)

In [19]:
tic = time.time()
pred_label, _ = recog_model.predict(X_test)
toc_test = time.time()-tic
toc_test_onetrial = toc_test/len(Y_test)

Finally, we can use the build-in functions to quickly calculate the classification accuracy and ITR.

In [20]:
from SSVEPAnalysisToolbox.evaluator import cal_acc,cal_itr
acc = cal_acc(Y_true = Y_test, Y_pred = pred_label)
itr = cal_itr(tw = tw, t_break = dataset.t_break, t_latency = dataset.default_t_latency, t_comp = toc_test_onetrial,
              N = len(freqs), acc = acc)
print("""
Simulation Information:
    Method Name: {:s}
    Dataset: {:s}
    Signal length: {:.3f} s
    Channel: {:s}
    Subject index: {:n}
    Testing block: {:s}
    Training block: {:s}
    Training time: {:.5f} s
    Total Testing time: {:.5f} s
    Testing time of single trial: {:.5f} s

Performance:
    Acc: {:.3f} %
    ITR: {:.3f} bits/min
""".format(recog_model.ID,
           dataset.ID,
           tw,
           str(ch_used),
           sub_idx,
           str(test_block_list),
           str(train_block_list),
           toc_train,
           toc_test,
           toc_test_onetrial,
           acc*100,
           itr))


Simulation Information:
    Method Name: eTRCA
    Dataset: Benchmark Dataset
    Signal length: 1.000 s
    Channel: [47, 53, 54, 55, 56, 57, 60, 61, 62]
    Subject index: 1
    Testing block: [0]
    Training block: [1, 2, 3, 4, 5]
    Training time: 0.08602 s
    Total Testing time: 1.27929 s
    Testing time of single trial: 0.03198 s

Performance:
    Acc: 97.500 %
    ITR: 180.186 bits/min

